In [113]:
#importamos librerias y archivo
import pandas as pd
import numpy as np
import re

In [114]:
#Leemos el dataset
data=pd.read_csv('properati.csv')

## Faltantes de Place_Name

In [115]:
#veamos vacios de place_name
mask_place_name_null = data.place_name.isnull()
data.place_with_parent_names[mask_place_name_null].value_counts()
#al ser todos los faltantes de tigre se los asignamos a place_name
data["place_name"]=data.apply(lambda x : 'Tigre' if x['place_name'] is np.NaN else x['place_name'], axis=1)

## Faltantes de Cuartos

In [116]:
#extraemos la cantidad de cuartos del campo title
pattern_room =("(?P<trae>\d*\s(?=dorm)|\d\s(?=Dorm)|\d\s(?=DORM)|\d*\s(?=AMB)|[a-zA-Z](?=ONOAMB)|\d*\s(?=amb)|[a-zA-Z](?=onoamb))*")
objeto_room= re.compile(pattern_room)
mask_pattern_room=data.title.apply (lambda x: x if x is np.NaN else objeto_room.match(x.lower()))
new_room = mask_pattern_room.apply(lambda x: x if x is np.NaN else x.group("trae"))
data["new_room"]=new_room.str.strip()
data["new_room"]=data["new_room"].replace ("m",1)
data["new_room"]=data["new_room"].replace ("M",1)
new_room.notnull().sum()

#extraemos la cantidad de cuartos del campo description
pattern_room_desc =("(?P<ambs>\d*\s(?=dorm)|\d*\s(?=amb)|[a-zA-Z](?=onoamb))*")
objeto_room_desc= re.compile(pattern_room_desc)
mask_pattern_room_desc=data.description.apply (lambda x: x if x is np.NaN else objeto_room_desc.match(x.lower()))
new_room_desc = mask_pattern_room_desc.apply(lambda x: x if x is np.NaN else x.group("ambs"))
data["new_room_desc"]=new_room.str.strip()
data["new_room_desc"]=data["new_room_desc"].replace ("m",1)
data["new_room_desc"]=data["new_room_desc"].replace ("M",1)
new_room_desc.notnull().sum()

#coalesce con las 3 columnas
data["new_room_final"]=np.NaN
data["new_room_final"]=data.new_room_final.fillna(data.rooms).fillna(data.new_room).fillna(data.new_room_desc)
data["new_room_final"]=pd.to_numeric(data["new_room_final"])

##Alternativa de Coalesce->nos quedamos con el valor final de rooms
#data["new_room_coalesce"]=data.apply(lambda x : x['rooms'] if x['rooms']>0 else x['new_room'], axis=1)
#data["new_room_coalesce"]=pd.to_numeric(data["new_room_coalesce"])
#data["new_room_final"]=data.apply(lambda x : x['new_room_coalesce'] if x['new_room_coalesce']>0 else x['new_room_desc'], axis=1)
#data["new_room_final"]=pd.to_numeric(data["new_room_final"])

## Faltantes de M2

In [117]:
#extraemos los metros cuadrados del campo title
pattern_m2="(?P<metros>\d*\.?\,?\d+\s?(?=m2)|\d*\.?\,?\d*\s?(?=mts)|\d*\.?\,?\d*\s?(?=m))*"
objeto_m2=re.compile(pattern_m2)
mask_pattern_m2=data.title.apply (lambda x: x if x is np.NaN else objeto_m2.match(x.lower()))
new_m2 = mask_pattern_m2.apply(lambda x: x if x is np.NaN else x.group("metros"))
data["new_m2"]=new_m2.str.strip()
new_m2.notnull().sum()

#extraemos los metros cuadrados del campo description
pattern_m2_desc="(?P<metros>\d*\.?\,?\d+\s?(?=metro)|\d*\.?\,?\d+\s?(?=m2)|\d*\.?\,?\d*\s?(?=mts)|\d*\.?\,?\d*\s?(?=m(\.|\,|\s)))*"
objeto_m2_desc=re.compile(pattern_m2_desc)
mask_pattern_m2_desc=data.description.apply (lambda x: x if x is np.NaN else objeto_m2_desc.match(x.lower()))
new_m2_desc = mask_pattern_m2_desc.apply(lambda x: x if x is np.NaN else x.group("metros"))
data["new_m2_desc"]=new_m2.str.strip()
new_m2_desc.notnull().sum()

#coalesce con las 3 columnas
data["new_m2_covered_final"]=np.NaN
data["new_m2_covered_final"]=data.new_m2_covered_final.fillna(data.surface_total_in_m2).fillna(data.new_m2).fillna(data.new_m2_desc)
data["new_m2_covered_final"]=pd.to_numeric(data["new_m2_covered_final"])

## Alternativa de coalesce-> nos quedamos con el valor final de m2
#data["new_m2_covered_final"]=data.apply(lambda x : x['surface_covered_in_m2'] if x['surface_covered_in_m2']>0 else x['new_m2_desc'], axis=1)
#data["new_m2_covered_final"]=pd.to_numeric(data["new_m2_covered_final"])

## Faltantes de Montos

In [118]:
#extraemos el valor de dolares del campo description
pattern_usd="(?P<usd>(?<=venta usd)\s?\d*\.?\d*|(?<=usd)\s?\d*\.?\d*|(?<=USD)\s?\d*)*"
objeto_usd=re.compile(pattern_usd)
mask_pattern_usd=data.description.apply(lambda x: x if x is np.NaN else objeto_usd.match(x.lower()))
new_usd = mask_pattern_usd.apply(lambda x: x if x is np.NaN else x.group("usd"))
data["new_usd"]=new_usd

#extraemos el valor de dolares del campo title
pattern_usd2="(?P<usd>(?<=venta usd)\s?\d*\.?\d*|(?<=usd)\s?\d*\.?\d*|(?<=USD)\s?\d*)*"
objeto_usd2=re.compile(pattern_usd2)
mask_pattern_usd=data.title.apply(lambda x: x if x is np.NaN else objeto_usd2.match(x.lower()))
new_usd_title = mask_pattern_usd.apply(lambda x: x if x is np.NaN else x.group("usd"))
data["new_usd_title"]=new_usd_title

#coalesce con las 3 columnas
data["usd_final"]=np.NaN
data["usd_final"]=data.usd_final.fillna(data.price_aprox_usd).fillna(data.new_usd).fillna(data.new_usd_title)
data["usd_final"]=pd.to_numeric(data["usd_final"])

data[["price_aprox_usd","new_usd","new_usd_title","usd_final"]].to_csv("USDFinal.csv")

## Split URL para las operaciones y tipo de propiedad

In [119]:
#split url para tener operaciones correctas y tipo de propiedad
splitvalues = data["properati_url"].str.split("_", expand = True) 
data["new_operacion"]=splitvalues[1]
data["new_tipo_prop"]=splitvalues[2]
data["new_tipo_prop"]=data["new_tipo_prop"].replace("departamentos","departamento")
#descomentar para ver salida splitvalues.to_csv('Spliturl.csv')

##Alternativa de split sin normatizar departamento(s)->tomamos la operacion y el tipo de propiedad de la url
#data["new_operacion"]=data.properati_url.str.split('_').str[1]
##data["new_operacion"].value_counts()
#data["new_tipo_prop"]=data.properati_url.str.split('_').str[2]
#data["new_tipo_prop"].value_counts()

## Dummies para amenities

In [120]:
#dummies para los ammenities
data.loc[data.description.str.contains(pat="((\.|\,|\s)(gym)|(\.|\,|\s)(gimnasio))", case=False, na=False, regex=True),'gym'] = 1
data.loc[~(data.description.str.contains(pat="((\.|\,|\s)(gym)|(\.|\,|\s)(gimnasio))", case=False, na=False, regex=True)),'gym'] = 0
data.loc[data.description.str.contains(pat="((\.|\,|\s)(sum)|(\.|\,|\s)(usos multiples)|(\.|\,|\s)(multiproposito))", case=False, na=False, regex=True),'sum'] = 1
data.loc[~(data.description.str.contains(pat="((\.|\,|\s)(sum)|(\.|\,|\s)(usos multiples)|(\.|\,|\s)(multiproposito))", case=False, na=False, regex=True)),'sum'] = 0
data.loc[data.description.str.contains(pat="((\.|\,|\s)(pileta)|(\.|\,|\s)(piscina))", case=False, na=False, regex=True),'pileta'] = 1
data.loc[~(data.description.str.contains(pat="((\.|\,|\s)(pileta)|(\.|\,|\s)(piscina))", case=False, na=False, regex=True)),'pileta'] = 0
data.loc[data.description.str.contains(pat="((\.|\,|\s)(spa))", case=False, na=False, regex=True),'spa'] = 1
data.loc[~(data.description.str.contains(pat="((\.|\,|\s)(spa))", case=False, na=False, regex=True)),'spa'] = 0
data.loc[data.description.str.contains(pat="((\.|\,|\s)(lavadero)|(\.|\,|\s)(loundry)|(\.|\,|\s)(laundry)|(\.|\,|\s)(lavado))", case=False, na=False, regex=True),'laundry'] = 1
data.loc[~(data.description.str.contains(pat="((\.|\,|\s)(lavadero)|(\.|\,|\s)(loundry)|(\.|\,|\s)(laundry)|(\.|\,|\s)(lavado))", case=False, na=False, regex=True)),'laundry'] = 0
data.loc[data.description.str.contains(pat="((\.|\,|\s)(balcon))", case=False, na=False, regex=True),'balcon'] = 1
data.loc[~(data.description.str.contains(pat="((\.|\,|\s)(balcon))", case=False, na=False, regex=True)),'balcon'] = 0
data.loc[data.description.str.contains(pat="((\.|\,|\s)(jardin)|(\.|\,|\s)(patio)|(\.|\,|\s)(parque))", case=False, na=False, regex=True),'patio'] = 1
data.loc[~(data.description.str.contains(pat="((\.|\,|\s)(jardin)|(\.|\,|\s)(patio)|(\.|\,|\s)(parque))", case=False, na=False, regex=True)),'patio'] = 0
data.loc[data.description.str.contains(pat="((\.|\,|\s)(terraza))", case=False, na=False, regex=True),'terraza'] = 1
data.loc[~(data.description.str.contains(pat="((\.|\,|\s)(terraza))", case=False, na=False, regex=True)),'terraza'] = 0
data.loc[data.description.str.contains(pat="((\.|\,|\s)(hidromasaje)|(\.|\,|\s)(hidro masaje))", case=False, na=False, regex=True),'hidromasaje'] = 1
data.loc[~(data.description.str.contains(pat="((\.|\,|\s)(hidromasaje)|(\.|\,|\s)(hidro masaje))", case=False, na=False, regex=True)),'hidromasaje'] = 0
percent_missing = data.isnull().sum() * 100 / len(data)
missing_value_df = pd.DataFrame({'column_name': data.columns,
                            'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', inplace=True, ascending=False)
missing_value_df

#descomentar para ver salida hasta este punto 
#data.to_csv("salida.csv")

C:\Users\gomez.gustavo\AppData\Local\Continuum\anaconda3\envs\dhdsblend_1\lib\site-packages\pandas\core\strings.py:1954: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,column_name,percent_missing
new_usd_title,new_usd_title,100.000000
new_usd,new_usd,100.000000
new_m2_desc,new_m2_desc,97.354397
new_m2,new_m2,97.354397
new_room_desc,new_room_desc,96.300115
new_room,new_room,96.300115
floor,floor,93.483749
expenses,expenses,88.234615
rooms,rooms,60.905791
new_room_final,new_room_final,59.627949


In [121]:
#Creando nueva columna para imputar las latitudes y longitudes. Adicionalmente se coloca todo el proceso de imputacion.
data["comb_place"] = data["place_name"] + data["state_name"]
#print(data["comb_place"].head(10))
latcomb_place = data[["comb_place", "lat"]].drop_duplicates(inplace = False)
loncomb_place = data[["comb_place", "lon"]].drop_duplicates(inplace = False)
latcomb_place.dropna(inplace = True)
loncomb_place.dropna(inplace = True)
latcomb_dict = dict(zip(latcomb_place["comb_place"], latcomb_place["lat"]))
loncomb_dict = dict(zip(loncomb_place["comb_place"], loncomb_place["lon"]))
data["lat_limpia"] = data.comb_place.map(latcomb_dict)
data["lon_limpia"] = data.comb_place.map(loncomb_dict)
#data["lat_limpia"].isnull().sum()
#data["lon_limpia"].isnull().sum()
data= data.drop(['lat','lon','comb_place'], axis=1)

In [122]:
#Revision Missing Values
percent_missing = data.isnull().sum() * 100 / len(data)
missing_value_df = pd.DataFrame({'column_name': data.columns,
                            'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', inplace=True, ascending=False)
missing_value_df


,column_name,percent_missing
new_usd_title,new_usd_title,100.000000
new_usd,new_usd,100.000000
new_m2,new_m2,97.354397
new_m2_desc,new_m2_desc,97.354397
new_room_desc,new_room_desc,96.300115
new_room,new_room,96.300115
floor,floor,93.483749
expenses,expenses,88.234615
rooms,rooms,60.905791
new_room_final,new_room_final,59.627949


## Eliminaciones y Dummies Finales

In [123]:
#eliminamos filas sin informacion de precios
data=data[(data['price'].notnull())|(data['price_aprox_usd'].notnull())|(data['price_aprox_local_currency'].notnull())|(data['price_per_m2'].notnull())]

#eliminamos las columnas que no vamos a usar por tener alto porcentaje de nulos 
data= data.drop(['lat-lon','image_thumbnail','floor','rooms','expenses','new_room','new_room_desc','new_m2','new_m2_desc'], axis=1)

#Eliminamos las filas sin precio final esto elimina GALPON LA PAMAPA Y FORMOSA DE LAS DUMMY
data=data[(data['usd_final'].notnull())]

#Eliminamos las filas sin numero de Rooms Final esto elimina GALPON LA PAMAPA Y FORMOSA DE LAS DUMMY
data=data[(data['new_room_final'].notnull())]

#checkeo vacios 
nulls = data.apply(lambda x: x.isnull().sum() , axis=0)

#Calculamos el porcentaje de nulos por total de filas
nulls_porcentaje= data.apply(lambda x: x.isnull().sum()*100/x.size,axis=0)

##descomentar para ver-> Ordenamos de mayor a menor por participacion de nulos
#print(nulls_porcentaje.sort_values(ascending=False))

#get dummies de place_name (Descartamos por alta dimensionalidad)
#data.place_name.str.get_dummies().head(10) 

#get dummies de state_name
data=pd.concat([data, data.state_name.str.get_dummies(),data.new_operacion.str.get_dummies(),data.new_tipo_prop.str.get_dummies()], axis=1)
#Eliminamos los ceros en price
mask_price_cero = data.price!=0
mask_price_cero.sum()
data=data[mask_price_cero]
#descomentar para ver salida hasta este punto 
#data.to_csv('salida.csv')
state_name_unique = data.state_name.unique()
state_name_unique
property_unique=data.property_type.unique()
property_unique
#data_grouped_state_name = data.groupby('state_name')
#data_grouped_state_name["price"].mean()
#data.pivot_table(index=state_name_unique, columns=property_unique)
pivot=data.pivot_table(values='price', index='state_name', columns='property_type',aggfunc='min')
pivot
print(data.operation.value_counts())
print(data.new_operacion.value_counts())
print(data.property_type.value_counts())
print(data.new_tipo_prop.value_counts())


sell    41064
Name: operation, dtype: int64
venta                40991
alquiler                72
alquiler-temporal        1
Name: new_operacion, dtype: int64
apartment    29266
house         8847
PH            2864
store           87
Name: property_type, dtype: int64
departamento           29246
casa                    8846
ph                      2842
local                     87
otro                      11
cochera                   10
terreno                    6
campo                      6
oficina-consultorio        5
edificio                   3
quinta                     1
loft                       1
Name: new_tipo_prop, dtype: int64


## Salida CSV

In [124]:
data[["new_room_final","new_m2_covered_final","usd_final","lat_limpia","lon_limpia","alquiler","alquiler-temporal","venta","campo","casa","cochera","departamento","edificio","local","loft","oficina-consultorio","otro","ph","quinta","terreno","gym","sum","pileta","spa","laundry","balcon","patio","terraza","hidromasaje","Bs.As. G.B.A. Zona Norte","Bs.As. G.B.A. Zona Oeste","Bs.As. G.B.A. Zona Sur","Buenos Aires Costa Atlántica","Buenos Aires Interior","Capital Federal","Catamarca","Chaco","Chubut","Corrientes","Córdoba","Entre Ríos","Jujuy","La Rioja","Mendoza","Misiones","Neuquén","Río Negro","Salta","San Juan","San Luis","Santa Cruz","Santa Fe","Santiago Del Estero","Tierra Del Fuego","Tucumán"]].to_csv("salida2.csv")
